<a href="https://colab.research.google.com/github/jamg-upv/LLMforSLRscreening/blob/main/ACEDE-ECNwsDic24/ACEDE_ECN_dic24_cribado_LLMhiwp_embeddingsv2_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este codigo funciona con un entorno de de computacion de 12GB RAM sin GPU.
Lo he probadocon 4 categorias y 1300 abstracts a clasificar y con modelos de embeedings de 384 tokens no gasta más de 3Gb de ram. Con los 14 modelos (donde hay algunos de 1024 tokens) llega a gastar 6Gb (la ram depende de los tokens del modelo y no de la cantidad de objetos a embedd)
los modelos de 384 tokens consumen unos 3 minutos por cada 1000 abstracts
Los modelosd e 720 tokens consumen unos 5 minutos

Si fuese necesario, me planteo compara bloques de computación para poder usar un GPU o entornos más intensivos de RAM

#Preparacion previa (ejecutar solo una vez)



## (solo ejecutar si tus datos están en github y quieres importarlos) conexion a github

In [3]:
#conectar con repositorio github publico para acceso a los datasets (ajusta el nombre del repositorio al que corresponda)
!git clone https://github.com/jamg-upv/LLMforSLRscreening.git

# elije la carpeta del repositorio donde esta el archivo de datos   ACEDE-ECNwsDic24/
# input_path = '/content/LLMforSLRscreening/datasets/'
input_path = '/content/LLMforSLRscreening/ACEDE-ECNwsDic24/'

# Define el nombre del archivo #Receuerda: estructura del archivo de datos:
# Component	Description	Type	Class
input_file_name = 'ACEDE-ECN-workshop-dic-2024-HIWP3.csv'


fatal: destination path 'LLMforSLRscreening' already exists and is not an empty directory.


## (Solo ejecutar si quieres guardar los resultados en TU googel drive)  googledrive montado en local

In [4]:
#conecar a google drive para guardar alli los outputs que quiera mantener (y que no se me olvide descargarlos) luego los subiré a Git para su uso posterior
from google.colab import drive
import os

# Montar Google Drive si aún no está montado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define el path de salida en Google Drive
output_path = '/content/drive/MyDrive/Reto21dias_24/'

# Verifica si el directorio existe y si no, lo crea
if not os.path.exists(output_path):
    try:
        os.makedirs(output_path)
        print(f"Directorio creado: {output_path}")
    except Exception as e:
        print(f"Error al crear el directorio: {e}")
else:
    print(f"El directorio ya existe: {output_path}")

Mounted at /content/drive
El directorio ya existe: /content/drive/MyDrive/Reto21dias_24/


##Ruta a los archivos cargados y guardados en raiz de Colab (si se corta la conexion sin descargarlos se pierden)

In [ ]:
# Define el nombre del archivo y el path de entrada y salida en raiz de google colab

# estructura del archivo de datos input_file: # Component	Description	Type	Class
# input_file_name = 'ACEDE-ECN-workshop-dic-2024-HIWP3.csv'
input_file_name = 'data.xlsx'

input_path = '/content/'

output_path = '/content/'

Parametrización de variables

In [ ]:
import os # Import the os module at the top of your script
# Construye el path completo
full_in_path = os.path.join(input_path, input_file_name)
full_in_path
# full_out_path = os.path.join(output_path, file_name)

# Define la variable con el nombre de la columna
target_column = 'Description'

# Objetos a clasificar
selec_objects = ['art']
# selec_objects = ['cat1']

#categorias en la que quiero clasificar los objetos
selec_categories = ['cat1','cat2']
# selec_categories = ['cat2', 'cat3','cat4']

# el identificador [component] de la categoria que quiero mostrar en el resumen de clasificación global
# target_category ='HIWPshortDescrip'
# target_category ='HIWPLongDescrip'
target_category ='Empowerment'
# target_category ='WorkLifeBalance'
# target_category ='RemoteWork'
# target_category ='GrenHRM'
# target_category ='KaizenLong'
# target_category ='KaizenCulture'

# etiquetar los archivos con alguna información adcional
# tag_file = ''
# tag_file = 'all models-articles'
tag_file = '1 modelo rapido'

# prueba (en desarrollo) para leer diferentes formatos de csv (no comprobada)

In [ ]:
import pandas as pd

def leer_archivo(ruta_archivo, **kwargs):
    extension = ruta_archivo.split('.')[-1].lower()

    try:
        if extension in ['xlsx', 'xls']:
            return pd.read_excel(ruta_archivo, **kwargs)
        elif extension == 'csv':
            # Lista de configuraciones a intentar
            configs = [
                {'encoding': 'utf-8', 'sep': ','},
                {'encoding': 'latin1', 'sep': ';'},
                {'encoding': 'utf-8', 'sep': ';'},
                {'encoding': 'cp1252', 'sep': ';'}  # Común en Windows
            ]

            # Intentar cada configuración
            for config in configs:
                try:
                    # Combinar configuración predeterminada con kwargs
                    params = {**config, **kwargs}
                    return pd.read_csv(ruta_archivo, **params)
                except:
                    continue

            raise ValueError("No se pudo leer el archivo CSV con ninguna configuración")
        else:
            raise ValueError(f"Formato de archivo no soportado: {extension}")

    except Exception as e:
        print(f"Error al leer el archivo: {str(e)}")
        return None

# Ejemplos de uso:
# Lectura básica
hiwp_data = leer_archivo(full_in_path)

# Lectura con parámetros específicos
# hiwp_data = leer_archivo(full_in_path, decimal=',', thousands='.')
# hiwp_data = leer_archivo(full_in_path, sheet_name='Hoja1')  # Para Excel

# leer el archivo

In [ ]:
import pandas as pd
# Leer el archivo CSV o EXCEL
# Originalmente era un CSV de Open Office porque si lo generaba con MSexcel no se mapeaba bien )
    # Para guardar correctamente un archivo CSV con comas desde OpenOffice cuando viene de Excel con punto y coma (;), sigue estos pasos:
    # En OpenOffice, ve a "Archivo → Guardar como"
    # Selecciona "Texto CSV (.csv)" como tipo de archivo
    # Marca la casilla "Editar configuración de filtro"
    # En la ventana de configuración:
    # Juego de caracteres: Unicode (UTF-8)
    # Delimitador de campo: selecciona "," (coma)
    # Delimitador de texto: " (comillas)
    # NO se marca tener todas las cadenas de texto con ""
#mejoro el codigo con esta funcion que debe gestionar automático el formato de CSV o de excel (la primera de las hojas del libro)

def leer_archivo(ruta_archivo):
    # Obtener la extensión del archivo
    extension = ruta_archivo.split('.')[-1].lower()

    try:
        if extension in ['xlsx', 'xls']:
            # Leer archivo Excel
            return pd.read_excel(ruta_archivo)
        elif extension == 'csv':
            # Intentar primero con encoding UTF-8 y separador ','
            try:
                return pd.read_csv(ruta_archivo, encoding='utf-8')
            except:
                # Si falla, intentar con encoding latin1 y separador ';'
                try:
                    return pd.read_csv(ruta_archivo, encoding='latin1', sep=';')
                except:
                    # Último intento con encoding utf-8 y separador ';'
                    return pd.read_csv(ruta_archivo, encoding='utf-8', sep=';')
        else:
            raise ValueError(f"Formato de archivo no soportado: {extension}")

    except Exception as e:
        print(f"Error al leer el archivo: {str(e)}")
        return None

# Uso de la funcion
hiwp_data = leer_archivo(full_in_path)

# Verificar si la lectura fue exitosa
if hiwp_data is not None:
    print("Archivo leído correctamente")
    print(f"Dimensiones del DataFrame: {hiwp_data.shape}")
else:
    print("No se pudo leer el archivo")

# Mostrar el DataFrame resultante
# print(hiwp_data)

# Crear el DataFrame 'objects'
objects = hiwp_data[hiwp_data['Type'].isin(selec_objects)]

# Crear el DataFrame 'categories'
categories = hiwp_data[hiwp_data['Type'].isin(selec_categories)]

# Opción 1: Reconstruir el índice descartando el índice original
# objects= objects.reset_index(drop=True)
# categories= categories.reset_index(drop=True)

# Opción 2: Reconstruir el índice manteniendo el índice original como una nueva columna
objects= objects.reset_index()
categories= categories.reset_index()


# Verificar los resultados
print("DataFrame 'objects':")
print(objects.shape)
print(objects['Type'].value_counts())
print("\nPrimeras filas de 'objects':")
print(objects.head())

print("\n" + "="*50 + "\n")

print("DataFrame 'categories':")
print(categories.shape)
print(categories['Type'].value_counts())
print("\nPrimeras filas de 'categories':")
print(categories.head())

Archivo leído correctamente
Dimensiones del DataFrame: (244, 3)
DataFrame 'objects':
(242, 4)
Type
art    242
Name: count, dtype: int64

Primeras filas de 'objects':
   index                         Component  \
0      2                10.3390/su13148053   
1      3         10.1007/s11783-023-1709-x   
2      4  10.1590/s0104-403620220003003342   
3      5            10.22395/rium.v19n36a7   
4      6     10.1080/2331186X.2020.1840009   

                                         Description Type  
0  Teaching and Learning in Survival Mode: Studen...  art  
1  Self-sustained catalytic combustion of CO enha...  art  
2  Acceptance and resistance to e-learning adopti...  art  
3  Diseño de una herramienta guía para la impleme...  art  
4  The specific academic learning self-efficacy a...  art  


DataFrame 'categories':
(2, 4)
Type
cat1    1
cat2    1
Name: count, dtype: int64

Primeras filas de 'categories':
   index      Component                                        Description  \
0 


# Embeedings




In [1]:
from importlib import metadata
import subprocess
import sys
import importlib

required_packages = [
    'seaborn',
    'matplotlib',
    'sentence-transformers',
    'pandas',
    'numpy',
    'transformers',
    'torch',
    'xlrd',
    'openpyxl'
]

def get_installed_packages():
    """Obtiene una lista de paquetes instalados usando importlib.metadata"""
    try:
        return {dist.metadata['Name'].lower() for dist in metadata.distributions()}
    except Exception:
        return set()

def install_packages():
    installed_packages = get_installed_packages()
    packages_to_install = [
        pkg for pkg in required_packages
        if pkg.lower().replace('-', '_') not in installed_packages
    ]

    if packages_to_install:
        print("Instalando paquetes faltantes...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages_to_install)
            print("\nTodos los paquetes faltantes han sido instalados.")
        except subprocess.CalledProcessError as e:
            print(f"\nError durante la instalación: {e}")
            return False
    else:
        print("Todos los paquetes requeridos ya están instalados.")

    return True

def verify_installations():
    print("\nVerificando instalaciones:")
    all_installed = True
    for package in required_packages:
        try:
            # Intentar importar el módulo
            importlib.import_module(package.replace('-', '_'))
            # Obtener la versión instalada
            version = metadata.version(package)
            print(f"✓ {package} (versión {version})")
        except (ImportError, metadata.PackageNotFoundError):
            print(f"✗ {package} - No se pudo importar")
            all_installed = False
    return all_installed

def main():
    # Ejecutar instalación y verificación
    if install_packages() and verify_installations():
        print("\nProcediendo con las importaciones...")
        try:
            # Importar las bibliotecas necesarias
            from sentence_transformers import SentenceTransformer, util
            import pandas as pd
            import numpy as np
            import time
            import torch
            print("\nTodas las importaciones completadas con éxito.")
            return True
        except ImportError as e:
            print(f"\nError al importar las bibliotecas: {e}")
            return False
    else:
        print("\nHubo problemas con la instalación o verificación de paquetes.")
        return False

if __name__ == "__main__":
    main()

Instalando paquetes faltantes...

Todos los paquetes faltantes han sido instalados.

Verificando instalaciones:
✓ seaborn (versión 0.13.2)
✓ matplotlib (versión 3.8.0)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


✓ sentence-transformers (versión 3.2.1)
✓ pandas (versión 2.2.2)
✓ numpy (versión 1.26.4)
✓ transformers (versión 4.46.3)
✓ torch (versión 2.5.1+cu121)
✓ xlrd (versión 2.0.1)
✓ openpyxl (versión 3.1.5)

Procediendo con las importaciones...

Todas las importaciones completadas con éxito.


In [ ]:
from datetime import datetime
# Obtener la fecha y hora actual para etiquear hora del analisis en los nombres de archivo
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:

# Lista de modelos a probar
# Seleccion de los 1 de los rapidos entre los 5 mejores del articulo WPOM 2024
models_to_test = [
    'all-MiniLM-L6-v2'
]

# todos los modelos
# models_to_test = [
#     'all-MiniLM-L6-v2',
#     'all-distilroberta-v1',
#     'all-mpnet-base-v2',
#     'paraphrase-multilingual-mpnet-base-v2',
#     'distiluse-base-multilingual-cased-v1',
#     'all-MiniLM-L12-v2',
#     'allenai-specter',
#     'allenai/scibert_scivocab_uncased',
#     'distilbert-base-nli-mean-tokens',
#     'roberta-base-nli-stsb-mean-tokens',
#     'distiluse-base-multilingual-cased-v2',
#     'paraphrase-multilingual-MiniLM-L12-v2',
#     'stsb-roberta-large',
#     'bert-base-nli-mean-tokens'
# ]

# Seleccion de los 5 mejores del articulo WPOM 2024
# models_to_test = [
#     'all-MiniLM-L6-v2',
#     'all-distilroberta-v1',
#     'all-mpnet-base-v2',
#     'all-MiniLM-L12-v2',
#     'allenai-specter'
# ]

# Seleccion de los peores del articulo WPOM 2024
# models_to_test = [
#     'paraphrase-multilingual-mpnet-base-v2',
#     'distiluse-base-multilingual-cased-v1',
#     'allenai/scibert_scivocab_uncased',
#     'distilbert-base-nli-mean-tokens',
#     'roberta-base-nli-stsb-mean-tokens',
#     'distiluse-base-multilingual-cased-v2',
#     'paraphrase-multilingual-MiniLM-L12-v2',
#     'stsb-roberta-large',
#     'bert-base-nli-mean-tokens'
# ]

# Función para evaluar un modelo
def evaluate_model(model_name, objects_df, categories_df):
    print(f"Evaluating model: {model_name}")
    model = SentenceTransformer(model_name)

    start_time = time.time()

    # Generar embeddings para objetos
    object_texts = objects_df[target_column].tolist()
    object_embeddings = model.encode(object_texts, show_progress_bar=True)

    # Generar embeddings para categorías
    category_texts = categories_df[target_column].tolist()
    category_embeddings = model.encode(category_texts, show_progress_bar=True)

    # Calcular similitud coseno
    similarity_matrix = util.cos_sim(object_embeddings, category_embeddings)

    # Encontrar las mejores coincidencias
    best_matches = np.argmax(similarity_matrix, axis=1)

    end_time = time.time()
    processing_time = end_time - start_time

    return {
        'model': model_name,
        'best_matches': categories_df.iloc[best_matches]['Component'].tolist(),
        'processing_time': round(processing_time, 2),  # Redondear a 2 decimales
        'embedding_size': object_embeddings.shape[1],
        'similarity_matrix': similarity_matrix,
        'object_embeddings': object_embeddings,
        'category_embeddings': category_embeddings
    }

# Evaluar todos los modelos
results = []
similarity_data = []
embeddings_data = []

for model_name in models_to_test:
    try:
        result = evaluate_model(model_name, objects, categories)
        results.append({
            'model': result['model'],
            'best_matches': result['best_matches'],
            'processing_time': result['processing_time'],
            'embedding_size': result['embedding_size']
        })

        # Preparar datos de similitud para este modelo
        for i, obj in enumerate(objects['Component']):
            row = [model_name, obj] + result['similarity_matrix'][i].tolist()
            similarity_data.append(row)

        # Preparar datos de embeddings para este modelo
        for i, obj in enumerate(objects['Description']):
            embeddings_data.append([model_name, objects['Component'].iloc[i], obj, result['object_embeddings'][i].tolist()])
        for i, cat in enumerate(categories['Description']):
            embeddings_data.append([model_name, categories['Component'].iloc[i], cat, result['category_embeddings'][i].tolist()])

    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)
df_results = df_results.rename(columns={'processing_time': 'processing_time_seconds'})
print(df_results)

# Crear DataFrame de similitud coseno
columns = ['Model', 'Object'] + categories['Component'].tolist()
df_similarity = pd.DataFrame(similarity_data, columns=columns)

# Crear DataFrame de embeddings
df_embeddings = pd.DataFrame(embeddings_data, columns=['Model', 'Component', 'Description', 'Embedding'])

# Crear la nueva tabla de similitud para HIWPshortDescription
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])


Evaluating model: all-MiniLM-L6-v2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

              model                                       best_matches  \
0  all-MiniLM-L6-v2  [OpMange, OpMange, GrenHRM, OpMange, WorkLifeB...   

   processing_time_seconds  embedding_size  
0                   161.96             384  


In [ ]:
###OPCIONAL##### SOLO EJECUTAR para reahacer el analisis de proximidad con otra categoria como target sin tener que repetir los embeedings
# el identificador [component] de la categoria que quiero mostrar en el resumen de clasificación global
target_category ='HIWPshortDescrip'
# target_category ='Empowerment'
# target_category ='WorkLifeBalance'
# target_category ='RemoteWork'
# target_category ='GrenHRM'

# Crear la nueva tabla de similitud para la target_category
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])




In [ ]:
# Calcula el rango de similitud para cada modelo individualmente, donde el rango 1 es el mejor (mayor similitud).
# Calcula el rango promedio para cada Object_Description a través de todos los modelos.
# Ordena los objetos por su rango promedio (el más bajo es el mejor).
# La tabla resumen final contendrá las siguientes columnas:
# # Object_Component: El componente del objeto.
# # Object_Description: La descripción del objeto.
# # Average_Rank: El rango promedio del objeto a través de todos los modelos (más bajo es mejor).
# Los objetos estarán ordenados por su rango promedio,
# lo que  dará una visión integrada de cómo se comportan los objetos a clasificar, a través de todos los modelos,
# en relación con la categoria sobre la que me interesa clasificarlos.

# No es sencillo poner un punto de corte a partir del cual se consideran "excluidos" los objetos a clasificar.
# de modo que se haría un escreening por titulo y abstract manual.
# partiendo del orden de esta tabla hasta llegar aun  punto donde # haya varios seguidos sin seleccionar.
# Momento en el que se podría parar el screening asistido por clasificacion automática.

# Función para calcular el rango inverso (el más alto obtiene el rango 1)
def inverse_rank(series):
    return series.rank(ascending=False, method='min')

# Calcular rangos para cada modelo
df_ranks = df_hiwp_similarity.groupby('Model').apply(lambda x: x.assign(Rank=inverse_rank(x['Similarity']))).reset_index(drop=True)

# Calcular el rango promedio para cada Object_Component a través de todos los modelos
df_summary = df_ranks.groupby('Object_Component')['Rank'].mean().reset_index()
df_summary = df_summary.rename(columns={'Rank': 'Average_Rank'})

# Añadir la Description correspondiente a cada Component
df_summary = df_summary.merge(objects[['Component', 'Description']], left_on='Object_Component', right_on='Component', how='left')

# Ordenar por rango promedio (ascendente, ya que el rango más bajo es mejor)
df_summary = df_summary.sort_values('Average_Rank')

# Reordenar las columnas
df_summary = df_summary[['Object_Component', 'Description', 'Average_Rank']]

# Mostrar las primeras filas de la tabla resumen
print(df_summary.head(20))

                  Object_Component  \
20               10.1002/hrm.22077   
78               10.1002/hrm.22142   
230  10.1080/09585192.2018.1539863   
66               10.1002/hrm.22130   
423  10.1080/09585192.2022.2078990   
478  10.1080/09585192.2022.2163464   
967        10.1108/PR-10-2019-0561   
87               10.1002/hrm.22151   
234  10.1080/09585192.2018.1547779   
909        10.1108/PR-08-2020-0609   
36               10.1002/hrm.22093   
156              10.1002/hrm.22229   
263  10.1080/09585192.2021.1878648   
479  10.1080/09585192.2022.2163855   
250  10.1080/09585192.2019.1674357   
936        10.1108/PR-09-2020-0663   
17               10.1002/hrm.22068   
96               10.1002/hrm.22160   
594  10.1080/09585192.2024.2373246   
332  10.1080/09585192.2021.1956564   

                                           Description  Average_Rank  
20   The challenge of increasing employees' well-be...           1.0  
78   High performance work systems and perceived or...     

<ipython-input-31-5e0be0c47d18>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranks = df_hiwp_similarity.groupby('Model').apply(lambda x: x.assign(Rank=inverse_rank(x['Similarity']))).reset_index(drop=True)


In [ ]:
# etiquetar los archivos con alguna información adcional
# tag_file = tag_file
# tag_file = 'best7 models'
# tag_file = 'worst7 models'

In [ ]:
# código para guardar todos los DataFrames en un único archivo Excel, con cada DataFrame en una hoja separada

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Asegurarse de que la carpeta de salida existe
os.makedirs(output_path, exist_ok=True)

# Crear el nombre del archivo Excel
excel_filename = f'hiwp_all_csv_classification_{current_time}_{target_category}_{tag_file}.xlsx'
excel_path = os.path.join(output_path, excel_filename)

# Crear un ExcelWriter object
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # Guardar cada DataFrame en una hoja separada
    df_results.to_excel(writer, sheet_name='model_comparison_results', index=False)
    df_similarity.to_excel(writer, sheet_name='tablas_similitud_coseno', index=False)
    df_embeddings.to_excel(writer, sheet_name='embeddings', index=False)
    df_hiwp_similarity.to_excel(writer, sheet_name='hiwp_similarity', index=False)
    df_summary.to_excel(writer, sheet_name='hiwp_similarity_summary', index=False)

print(f"Todos los resultados han sido guardados en {excel_path}")

IndexError: At least one sheet must be visible